# Feature Selection

In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, SelectKBest, mutual_info_classif, SelectPercentile

## Import Dataset

In [3]:
accidents = pd.read_csv('data/accidents.csv', index_col=0)
accidents.head(2)

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,at_intersection,at_stop_sign_or_traffic_light,is_day,is_evening,is_morning,is_night,is_day.1,is_evening.1,is_morning.1,is_night.1
0,JC343143,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,0,0,1,0,0,0,1,0,0,0
1,JC194776,03/21/2019 10:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,4.0,...,0,1,0,0,0,1,0,0,0,1


## Split Data and Determine Features to Assess
Selecting features solely based on training data.

In [11]:
X = accidents[['crash_month',
              'CRASH_HOUR',
              'is_weekend',
              'has_snow_ice',
              'involved_pedest_cycle',
              'at_intersection',
              'at_stop_sign_or_traffic_light',
              'is_day',
              'is_evening',
              'is_morning',
              'is_night',
              'is_day.1',
              'is_evening.1',
              'is_morning.1',
              'is_night.1']]

y = accidents['is_fatal_or_incap']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

## Mutual Information

In [ ]:
mutual_info_scores = mutual_info_classif(X_train, y_train)